# Reinforcement learning


**TODO: Was ist Reinforcement Learning, Was sind die Vorteile, Allgemeines, etc.**

## Implementierung 3
Um die verschiedenen Algorithmen des Reinforcement Learnings wieder auf unser Beispiel anwenden zu können, wird zunächst eine allgemeine Funktion definiert.

`transport_goods()` führt so oft eine Aktion, die von der gewünschten `action_method()` ausgewählt wird, aus, bis die Ware an ihrem Zielort eingetroffen ist. Nach dem Ausführen wird die Darstellung falls gewünscht aktualisiert. Die Konstante `MAX_VISITS_STATE` gibt an, wie oft ein Zustand besucht werden kann, bevor ausgehend von diesem Zustand statt der eigentlich gewünschten Aktion eine zufällige Aktion gestartet wird. Diese Randomisierung ist notwendig, um zu verhindern, dass sich Aktions-Schleifen bilden, in denen der Agent festhängt. Im idealen Fall besucht der Agenten einen State nur ein mal (Position zweimal, einmal mit Paktet, einmal ohne). Der Wert wird trotzdem höher angesetzt, um keine zu starke Einschränkung zu bieten.

Bei der Funktion handelt es sich auch um eine Funktion der Klasse `Transport`.

In [ ]:
MAX_VISITS_STATE = 10

In [ ]:
def transport_goods(self):
    position_truck, position_goods, position_goal = self.state
    while not self.done:    
        self.steps += 1
        state_num = state_to_state_num(self.state)
        action = self.action_method(state_num)
        if action not in possible_actions[position_truck]:
            action = random.sample(possible_actions[position_truck], 1)[0]
        
        value = self.reward_function(action)
        new_state = self.transition_function(action)
        if self.visited_states.count(new_state) > MAX_VISITS_STATE:
            action = random.sample(possible_actions[position_truck], 1)[0]
            value  = self.reward_function(action)
            new_state = self.transition_function(action)
        
        self.current_value += value
        self.state = new_state
        self.visited_states.append(self.state)
        position_truck, position_goods, position_goal = self.state
        
        if position_goods == position_goal: self.done = True
        if self. visualize:
            action_label.value = 'Action: ' + actions_description[action]
            update_canvas(self)
        if self.stepwise: time.sleep(0.1)
Transport.transport_goods = transport_goods
del transport_goods

Im folgenden werden alle möglichen Zustände in einer Liste gespeichert, sodass sie nummerierbar und damit eindeutig identifizierbar sind. Die nachfolgenden Funktionen `state_to_state_num()` und `state_num_to_state()` dienen dazu, zwischen den beiden Darstellungsweisen zu wechseln.

In [ ]:
states = []
for row in range (0, 6):
    for col in range (0, 6):
        for position_goods in range (0, 5):
            for position_goal in range (0, 4):
                states.append(((row, col), position_goods, position_goal))

In [ ]:
def state_to_state_num(state):
    return states.index(state)

def state_num_to_state(num):
    return states[num]

Um einen Vergleich zu haben, wie der Transport ohne Reinforcement Learning aussehen würde, wird die folgende Methode genutzt. Die Funktion `random_action()` gibt eine zufällige Aktion zurück. Sie nutzt also keinen Lerneffekt oder Trainingsdaten. Im weiteren Verlauf der Arbeit werden verschiedene Algorithmen zur Auswahl der Aktionen implementiert, die genau dies tun. In der abschließenden Evaluation wird dann zu sehen sein, wie sich die Nutzung dieser Daten auf die Effizienz des Transports auswirkt.

In [ ]:
def random_action(state_num):
    return random.randrange(0, 6, 1)

## Modell basiert

* Was bedeutet Modell basiert?

## Modell frei
* Was bedeutet Modell frei?

### Passives lernen vs. aktives lernen
Beides Arten von Reinforcement learning

Passiv:
* Agent hat die optimale Policy
* Agent weiß, was er machen muss
* Ziel: feste Abfolge von Aktionen ausführen, wie gut ist die optimale Policy?
* ADP, TD

Aktiv:
* Agent kennt die optimale Policy nicht
* Agent muss entscheiden, was er tun muss
* Ziel: handeln und optimale Policy lernen
* ADP mit exploration function, Q-learning


### (Adaptive dynamic programming (ADP) )


### Monte-Carlo-Methode

Die Monte-Carlo-Methode ist eine Methode zur Ermittlung der zu erwartenden Belohnung beim Reinforcement Learning. Das besondere bei Monte-Carlo ist, dass es sich um einen randomisierten Algorithmus handelt. Als Vorraussetzung werden Erfahrungswerte, also Trainingsdaten bestehend aus einer Folge von Zuständen, Aktionen und Belohnungen benötigt. Die Idee ist, aus diesen Daten den durchschnittlich erreichten Wert nach besuchen eines Zustands $s$ zu ermitteln. Dazu gibt es zwei Ansätze: "Every-Visit" und "First-Visit".

Bei "First-Visit" werden alle Belohnungen, die zwischen dem ersten Besuch des betrachteten Zustands $s$ und dem Endzustand auftreten, aufsummiert. Die so berechnetetn Werte für alle Episoden werden aufaddiert und durch die Anzahl der betrachteten Episoden geteilt.

**TODO: Beispielgraph einfügen**

Bei "Every-Visit" wird ein neuer Summations-Term für jedes Auftreten des Zustands $s$ erstellt. Das bedeutet man summiert zunächst alle Belohnung zwischen dem ersten Auftreten und dem Endzustand, addiert dazu die Summe der Belohnungen zwischen dem zweiten Auftreten und dem Endzustand und so weiter bis zur Addition der Summe der Belohnungen zwischen dem letzten Auftreten und dem Endzustand. Dies wird für alle Episoden wiederholt, aufaddiert und durch die Anzahl der Summations-Terme, also die Anzahl des Auftretens des Zustands $s$, dividiert.

**TODO: Beispielgraph einfügen**

(vgl. Gupta 2020, Sutton 2015)

In der Anwendung von Monte-Carlo wird eine Episode von einem zufälligen Zustand bis zum Endzustand mit zufälligen Aktionen durchgespielt und dabei Zustände, Aktionen und Belohnungen aufgezeichnet. Anschließend werden die Werte der Zustände nach einer der oben erläuterten Methode berechnet. Anschließend wird ein neuer zufälliger Zustand bestimmt und unter Ausnutzung der zuvor berechneten Werte bis zum Ende geführt. Die Werte werden nach dem Durchgang neu berechnet und gemittelt. Dies wird viele Male wiederholt. So ergibt sich die gewünschte Werte-Matrix Q.

Bei diesem Algorithmus ist nicht sichergestellt, dass jeder Zustand nach dem Training besucht wurde. Ein weiterer Nachteil ist, dass das Problem terminieren muss, bevor die Werte berechnet werden können. Es muss sich also zwingend um terminirende Probleme handeln.

## Implementierung 4

In [1]:
#TODO: Implementierung
def monte_carlo_action(state_num):
    return 

### Temporal-difference learning (TD-Learning)

* Agent passt Policy nach jeder Aktion auf Basis einer geschätzten erwarteten Belohnung an (nicht erst, wenn er die Belohnung erhält)
* Funktion $V$ bewertet Zustand $s_t$
* Mit jeder Iteration wird $V$ genauer, d.h. $V$ nähert sich $V^{\pi}$ an
* Lernrate: Wie gut passen neue Werte die Bewertungsfunktion $V$ an?

$V(s_t)\leftarrow(1-\alpha)V(s_t)+\alpha (r_{t+1}  \gamma V(s_{t+1}))$ mit Lernrate $\alpha$ und Discount-Faktor $\gamma$

- mit Gewichtung der neuen Aktionen
- Vorteil: neue Aktionen haben eine höhere Auswirkungen auf das Model, minimiert die Variazn im vergleich zu monte carlo

Ähnlich wie bei Monte Carlo, wird Temporal difference learning zur Berechnung der Q-Matrix verwendet. Bei jedem Durchlauf entscheidet sich der Agent für die Aktion, die zu dem State mit dem höchsten Reward führt. Im Anschluss wird der Wert mit Hilfe der Value-Funktion nach folgender Formel angepasst:
    
$V(s_t)\leftarrow(1-\alpha)V(s_t)+\alpha(r_{t+1} \gamma V(s_{t+1}))$

wobei gilt:
* $V(s_t)$ ist der Wert des aktuellen States
* $r_t+1$ ist der Reward, der im nächstem Schritt erhalten wird
* $\gamma$ ist die Gewichtung, wie hoch der Einfluss des Wertes in den Gesamtwert eingeht
* $\alpha$ ist die Lernrate

Die Matrix optimiert sich direkt nach jedem Trainings-Durchlauf. Im Gegensatz zu Monte Carlo werden die Rewards, mit einem Gewicht $\gamma$ versehen. Dadurch wird aktuelleren Werten eine höhere Wertigkeit im Endergebnis gegeben. Der Vorteil dabei ist, dass die Varianz minimiert wird. Ausreißer haben also kleinere Auswirkungen auf das Endergebnis.

**TODO: Beispielgraph einfügen**

## Implementierung 5

In [2]:
#TODO: Implementierung
def td_learning_action(state_num):
    return 

### Q-learning

* Variante von TD-learning

- Modellfreier R.L. Algorythmus um der Wert einer Aktion zu berechnen
- Benötigt keine Anpassung des Enviroments, nut Rewards und stochastische Transitionen
- Für einen Endlichen Markov Decision Prozess, findet Q-learning die Optimale Policy(Regel), um eine optimale Aktionsauswahl zu treffen, auf Basis von zufälligen Aktionen, falls man unendlich Zeit zur Verfügung hat.
- In eine Matrix len(States)xlen(Aktionen) werden die Rewards für eine Aktion im State eingetragen.
- Initial ist die Matrix mit 0 gefüllt
- Die Q-Funktion: Q: S x A -> R mappt eine Aktion und ein State auf eine Rationale zahl, die in der Matrix eingetragen wird
- Neue Werte werden mit einer höheren Gewichtung in der Matirx einberechnet (gamma, discount factor)
- in der Tabelle sind die Werte der letzen States aufsummiert eingetragen, also die Rewards der ganzen Weges

## Implementierung 6

Zunächst wird eine Tabelle mit einer Zeile für jeden Zustand und einer Spalte für jede Aktion angelegt. In jede Zelle wird zu Beginn der Wert Null geschrieben.

In [ ]:
q_table = np.zeros([len(states), len(actions)])

Die `q_function()` nimmt einen Zustand und eine Aktion sowie die Lernrate $\alpha$ und den Discount-Faktor $\gamma$. Darauf basierend berechnet sie mit Hilfe der `q_table` den neuen Wert für den Zustand und die Aktion und trägt diesen an der entsprechenden Stelle in der Tabelle ein. Zurückgegeben wird außerdem der nächste Zustand. Die Formel für die Berechnung des neuen Wertes lautet folgendermaßen:

\\[Q_{\texttt{new}}(s_t, a_t) \leftarrow Q_{\texttt{old}}(s_t, a_t) + \alpha \cdot (r_t + \gamma \cdot max_q(s{t+1}, a) - Q_{\texttt{old}}(s_t, a_t))\\]

mit:
* $s_t$, $s_{t+1} \in$ len(states)
* a $\in$ actions
* $Q_{\texttt{old}}(s_t, a_t)$ = Tabellenwert in Zeile $s_t$ und Spalte $a_t$
* $\alpha$ = Lernrate
* $\gamma$ = Discount-Faktor
* $r_t$ = Reward
* max_q = Schätzung des optimalen Zukunftswertes

Der Wert in der Klammer wird dabei auch als "temporal difference" bezeichnet.

In [ ]:
def q_function(state, action, alpha, gamma):
    transport  = Transport(state, stepwise = False, visualize = False)
    
    reward     = transport.reward_function(action)
    next_state = transport.transition_function(action)
    
    state_num      = state_to_state_num(state)
    next_state_num = state_to_state_num(next_state)
    
    old_value  = q_table[state_num][action]
    max_q      = np.max(q_table[next_state_num])
    new_value  = old_value + alpha * (reward + gamma * max_q - old_value)
    
    q_table[state_num, action] = new_value
    return next_state

Die Funktion `q_learning()` führt für eine zu übergebende Anzahl an Startzuständen (`episodes`) den Transport der Waren zum Ziel durch. Der Startzustand ist dabei ein zufälliger. Dies dient dem Training und somit der Verbesserung der `q_table`. Denn in jedem Schritt wird `q_function()` aufgerufen.  In rund 10% (= `epsilon`) der Schritte ist auch die Aktion zufällig ausgewählt. In den anderen Fällen wird die am besten bewertete Aktion für den aktuellen Zustand genutzt.

In [ ]:
def q_learning(episodes):
    alpha   = 0.1
    gamma   = 0.6
    epsilon = 0.1
    
    for episode in range(episodes):
        state = states[random.randrange(0, len(states), 1)]
        done  = False
        
        while not done:
            state_num = state_to_state_num(state)
            if random.uniform(0, 1) < epsilon:
                action = random.sample(possible_actions[state[0]], 1)[0] # Aktionsbereich erkunden
            else:
                action = np.argmax(q_table[state_num]) # Gelernte Werte ausnutzen
                
            state = q_function(state, action, alpha, gamma)
            _, position_goods, position_goal = state
            if position_goods == position_goal: done = True
        print(round(episode + 1/episodes*100, 1), '%', end='\r')

Zum Trainieren muss die Funktion `q_learning()` mit einem relativ hohen Wert aufgerufen werden. Damit dies nicht nach jedem Neustart des Kernels notwendig ist, kann die Tabelle abgespeichert und beim nächsten mal einfach gelade werden.

In [ ]:
#q_learning(10000) # Trainieren
#np.savetxt('q_table.txt', q_table) # Speichern der Tabelle
q_table = np.loadtxt('q_table.txt') # Laden der Tabelle

Die folgende Methode kann dann nach dem Trainieren in der tatsächlichen Anwendung genutzt werden, üm für jeden Zustand die beste Aktion zu ermitteln. Dazu muss sie der Klasse `Transport` bei der Initialisierung übergeben werden und wird später dann von `transport_goods()` aufgerufen.

In [ ]:
def q_learning_action(state_num):
    return np.argmax(q_table[state_num]) 